In [2]:
import re
import unicodedata
import pandas as pd
import nltk
from wordcloud import WordCloud
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from googletrans import Translator

In [5]:
translator = Translator()

translation = translator.translate("مرحبا كيف حالك؟",  dest='en')
translation.text

'Hello how are you?'

In [3]:
trigram = pd.read_csv('all_trigrams.csv')
bigram = pd.read_csv('all_bigrams.csv')

In [6]:
bigram.sort_values('overall', ascending = False).head(25)

,bigram,alittihad,sabanews,echoroukonline,alqabas,almasryalyoum,techreen,ryiadh,almustaqbal,saudiyoum,youm7,overall
0,"('الولايات', 'المتحده')",21937,2756,5112,18347,14258,26738,21934,31794,39069,41896,223851
1,"('الشرق', 'الاوسط')",4651,399,1161,4603,4992,5139,6269,9311,9156,14601,60292
2,"('الامم', 'المتحده')",4930,698,1324,3565,2846,7655,4525,8339,9615,8726,52233
3,"('باراك', 'اوباما')",6294,1030,1461,2304,3076,3593,3678,5414,4519,18690,50069
4,"('في', 'العراق')",0,0,0,0,0,0,8505,17149,16514,6699,48877
5,"('الرئيس', 'الامريكي')",182,766,1592,244,5892,1031,4301,0,10442,20195,44655
6,"('جورج', 'بوش')",2098,29,785,3022,2791,6559,4112,9521,8996,3918,41841
7,"('في', 'مصر')",0,0,0,0,0,0,1464,1168,1747,36474,40863
8,"('تنظيم', 'القاعده')",4790,1141,3034,2895,2519,2695,3672,6343,5269,8245,40613
9,"('البيت', 'الابيض')",4340,575,997,2832,2965,4866,3788,5753,6099,7865,40090


In [7]:
bigram.sort_values('techreen', ascending = False).head(25)

,bigram,alittihad,sabanews,echoroukonline,alqabas,almasryalyoum,techreen,ryiadh,almustaqbal,saudiyoum,youm7,overall
0,"('الولايات', 'المتحده')",21937,2756,5112,18347,14258,26738,21934,31794,39069,41896,223851
2,"('الامم', 'المتحده')",4930,698,1324,3565,2846,7655,4525,8339,9615,8726,52233
11,"('الرئيس', 'الاميركي')",6980,230,191,4051,0,6980,2854,12509,2257,0,36062
6,"('جورج', 'بوش')",2098,29,785,3022,2791,6559,4112,9521,8996,3918,41841
40,"('الاداره', 'الاميركيه')",1943,47,51,1664,0,6044,739,6003,702,0,17203
10,"('مجلس', 'الامن')",3214,519,860,3732,2051,5676,3040,7771,6458,4685,38016
1,"('الشرق', 'الاوسط')",4651,399,1161,4603,4992,5139,6269,9311,9156,14601,60292
9,"('البيت', 'الابيض')",4340,575,997,2832,2965,4866,3788,5753,6099,7865,40090
3,"('باراك', 'اوباما')",6294,1030,1461,2304,3076,3593,3678,5414,4519,18690,50069
71,"('الاميركي', 'جورج')",1238,6,44,1637,0,3456,928,5666,1033,0,14018


In [9]:
bigram.sort_values('sabanews', ascending = False).head(25)

,bigram,alittihad,sabanews,echoroukonline,alqabas,almasryalyoum,techreen,ryiadh,almustaqbal,saudiyoum,youm7,overall
0,"('الولايات', 'المتحده')",21937,2756,5112,18347,14258,26738,21934,31794,39069,41896,223851
8,"('تنظيم', 'القاعده')",4790,1141,3034,2895,2519,2695,3672,6343,5269,8245,40613
3,"('باراك', 'اوباما')",6294,1030,1461,2304,3076,3593,3678,5414,4519,18690,50069
101,"('القوات', 'المسلحه')",1349,967,185,576,1186,674,828,787,1404,3945,11911
7598,"('الاخ', 'الرئيس')",0,797,0,0,0,0,0,0,0,0,807
5,"('الرئيس', 'الامريكي')",182,766,1592,244,5892,1031,4301,0,10442,20195,44655
1066,"('الليله', 'الماضيه')",1311,717,39,350,59,441,0,0,0,697,3624
2,"('الامم', 'المتحده')",4930,698,1324,3565,2846,7655,4525,8339,9615,8726,52233
4489,"('منصور', 'هادي')",430,674,0,191,46,0,0,0,0,0,1351
14,"('المتحده', 'الامريكيه')",91,644,1668,2000,2234,1410,5505,0,5353,9059,27974


In [12]:
text_to_translate = str()

In [11]:
regex_exp = r'(.+)', '(.+)'

In [14]:
# word1, word2 = re.search(regex_exp, text_to_translate)

In [15]:
trigram.sort_values('techreen', ascending = False).head(25)

,index,trigram,alittihad,sabanews,echoroukonline,alqabas,almasryalyoum,techreen,ryiadh,almustaqbal,saudiyoum,youm7,overall
2132,3225,"('الاميركي', 'جورج', 'بوش')",927,0,36,1488,0,3150,876,5289,981,0,12747
2708,6845,"('الرئيس', 'الاميركي', 'جورج')",908,0,31,1402,0,2872,853,5003,919,0,11988
14920,37700,"('من', 'جيشنا', 'الباسل')",0,0,0,0,0,2122,0,0,0,0,2122
5461,13042,"('الولايات', 'المتحده', 'الاميركيه')",1287,70,20,1305,0,2096,249,2509,242,0,7778
1998,5386,"('الحرب', 'علي', 'العراق')",78,0,38,174,188,1753,562,838,986,403,5020
16,34364,"('مجلس', 'الامن', 'الدولي')",1129,207,263,798,383,1533,790,1309,1905,1385,9702
234,6959,"('الرئيس', 'بشار', 'الاسد')",357,17,122,657,135,1526,213,430,246,509,4212
1032,41392,"('وزيره', 'الخارجيه', 'الاميركيه')",1538,61,58,1055,0,1471,523,2751,396,99,7952
372,3221,"('الاميركي', 'باراك', 'اوباما')",3518,169,102,1072,0,1448,835,2740,456,147,10487
450,6842,"('الرئيس', 'الاميركي', 'باراك')",3273,155,92,975,0,1318,788,2544,431,140,9716


In [16]:
trigram.sort_values('sabanews', ascending = False).head(25)

,index,trigram,alittihad,sabanews,echoroukonline,alqabas,almasryalyoum,techreen,ryiadh,almustaqbal,saudiyoum,youm7,overall
6,2891,"('الامريكي', 'باراك', 'اوباما')",117,579,628,197,2063,411,725,0,1765,11932,18417
25933,10229,"('القوات', 'المسلحه', 'والامن')",70,556,0,0,0,0,0,0,0,0,626
7,6820,"('الرئيس', 'الامريكي', 'باراك')",112,538,585,183,1903,362,681,0,1635,11161,17160
2,13027,"('الولايات', 'المتحده', 'الامريكيه')",67,505,1258,1980,1734,1192,4478,0,4249,6920,22383
13419,24618,"('عبدربه', 'منصور', 'هادي')",340,362,0,72,33,0,42,0,0,0,849
5357,19227,"('تنظيم', 'القاعده', 'الارهابي')",107,324,20,48,0,211,145,62,150,0,1067
18464,33614,"('لوكاله', 'الانباء', 'اليمنيه')",47,302,0,0,0,0,0,0,0,0,349
11553,24400,"('عبد', 'ربه', 'منصور')",74,301,0,110,0,32,54,51,60,203,885
35857,22058,"('ربه', 'منصور', 'هادي')",69,301,0,105,0,0,48,47,62,194,826
15451,38565,"('منصور', 'هادي', 'رئيس')",0,299,0,0,0,0,0,0,0,0,299
